# 1. Import libraries


In [1]:
import random
from IPython.display import clear_output

# 2. Define players

In [14]:
class Player:

    @staticmethod
    def show_board(board):
        # Show the board
        print('|'.join(board[6:9]))
        print('|'.join(board[3:6]))
        print('|'.join(board[0:3]))


# Create human player
class HumanPlayer(Player):
    # Inherit Player class

    def reward(self, value, board):
        # Human player does not care about reward
        pass

    def make_move(self, board):
        # Human has to put a valid input; Try until the input is valid
        while True:
            try:
                self.show_board(board)            
                clear_output(wait=True)
                move = input('Your next move (cell index 1-9):')
                move = int(move)
                if not (move - 1 in range(9)) or board[move-1]!=' ' :
                    raise ValueError
            except ValueError:
                print('Invalid move; try again:\n')
            else:
                return move - 1


# Create AI player
class AIPlayer(Player):

    def __init__(self, epsilon=0.4, alpha=0.3, gamma=0.9, default_q=1):
        # epsilon: probability of exploration
        self.EPSILON = epsilon
        # learning rate
        self.ALPHA = alpha
        # discount parameter
        self.GAMMA = gamma
        # given state is not defined yet
        self.DEFAULT_Q = default_q
        # dictionary of states Q(s,a): return a reward for a state-action pair
        self.q = {}
        # previous move
        self.move = None
        # board in previous iteration
        self.board = (' ',) * 9

    def available_moves(self, board):
        # empty cells on the board so far
        return [i for i in range(9) if board[i] == ' ']

    def get_q(self, state, action):
        # return Q value, and create a new one if it is not in the dictionary yet
        if self.q.get((state, action)) is None:
            self.q[(state, action)] = self.DEFAULT_Q
        return self.q[(state, action)]

    def make_move(self, board):
        # make a random move with epsilon probability or pick the action with the highest Q value
        self.board = tuple(board)
        # retrieve available moves
        actions = self.available_moves(board)

        if random.random() < self.EPSILON:
            # exploration
            self.move = random.choice(actions)
        else:
            # exploitation
            q_values = [self.get_q(self.board, a) for a in actions]
            max_q_value = max(q_values)
            # if there are multiple best actions, choose one at random
            if q_values.count(max_q_value) > 1:
                best_actions = [i for i in range(len(actions)) if q_values[i] == max_q_value]
                best_move = actions[random.choice(best_actions)]
            else:
                best_move = actions[q_values.index(max_q_value)]
            self.move = best_move
        return self.move

    def reward(self, reward, board):
        # update Q(s,a)
        if self.move:
            prev_q = self.get_q(self.board, self.move)
            max_q_new = max([self.get_q(tuple(board), a) for a in self.available_moves(self.board)])
            self.q[(self.board, self.move)] = prev_q + self.ALPHA * (reward + self.GAMMA * max_q_new - prev_q)


In [25]:
class TicTacToe:
    BLANK = ' '
    AI_PLAYER = 'X'
    HUMAN_PLAYER = '0'
    REWARD_WIN = 10
    REWARD_LOSE = -10
    REWARD_TIE = -1

    # implement game logic
    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self.isHuman = isinstance(player1, HumanPlayer) or isinstance(player2, HumanPlayer)
        # randomly choose which player starts first
        self.first_player_turn = random.choice([True, False])
        self.board = [' '] * 9

    def play(self):
        # this is the game loop
        while True:
            if self.first_player_turn:
                player = self.player1
                other_player = self.player2
                player_tickers = (self.AI_PLAYER, self.HUMAN_PLAYER)
            else:
                player = self.player2
                other_player = self.player1
                player_tickers = (self.HUMAN_PLAYER, self.AI_PLAYER)

            # check game (win, lose, draw)
            game_over, winner = self.is_game_over(player_tickers)
            # game over: handle rewards
            if game_over:
                if winner == player_tickers[0]:
                    player.reward(self.REWARD_WIN, self.board[:])
                    other_player.reward(self.REWARD_LOSE, self.board[:])
                    if self.isHuman:
                        other_player.show_board(self.board)
                        print('\n %s won!' % player.__class__.__name__)
                elif winner == player_tickers[1]:
                    player.reward(self.REWARD_LOSE, self.board[:])
                    other_player.reward(self.REWARD_WIN, self.board[:])
                    if self.isHuman:
                        other_player.show_board(self.board)
                        print('\n %s won!' % other_player.__class__.__name__)
                else:
                    player.reward(self.REWARD_TIE, self.board[:])
                    other_player.reward(self.REWARD_TIE, self.board[:])
                    if self.isHuman:
                        other_player.show_board(self.board)
                        print('Tie!')
                break
            # next player's turn   
            self.first_player_turn = not self.first_player_turn
            # actual player's best move ( based on Q(s, a) table of AI player)
            move = player.make_move(self.board)        
            self.board[move] = player_tickers[0]

    def is_game_over(self, player_tickers):
        # consider both players
        for player_ticker in player_tickers:

            # check horizontal dimension
            for i in range(3):
                if self.board[3 * i + 0] == player_ticker and \
                        self.board[3 * i + 1] == player_ticker and \
                        self.board[3 * i + 2] == player_ticker:
                    return True, player_ticker
            # check vertical dimension
            for j in range(3):
                if self.board[j + 0] == player_ticker and \
                        self.board[j + 3] == player_ticker and \
                        self.board[j + 6] == player_ticker:
                    return True, player_ticker
            # check diagonals
            if self.board[0] == player_ticker and self.board[4] == player_ticker and self.board[8] == player_ticker:
                return True, player_ticker
            if self.board[2] == player_ticker and self.board[4] == player_ticker and self.board[6] == player_ticker:
                return True, player_ticker

        # return draw
        if self.board.count(' ') == 0:
            return True, None
        else:
            return False, None


In [32]:
TRAINING_EPOCHS = 100000
TRAINING_EPSILON = 0.2

# train AI player
ai_player_1 = AIPlayer()
ai_player_2 = AIPlayer()
print('Training the AI players...')
ai_player_1.EPSILON = TRAINING_EPSILON
ai_player_2.EPSILON = TRAINING_EPSILON

# training
for _ in range(TRAINING_EPOCHS):
    game = TicTacToe(ai_player_1, ai_player_2)
    game.play()

print('Training is Done')


Training the AI players...
Training is Done


In [40]:
# After training, set exploration to 0, the AI player will always play optimally
ai_player_1.EPSILON = 0
human_player = HumanPlayer()
game = TicTacToe(ai_player_1, human_player)
game.play()



Your next move (cell index 1-9):6
X|0|X
X|0|0
0|X|X
Tie!


#### 